In [1]:
import pandas as pd
import mysql.connector
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report


from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer



In [2]:
import pandas as pd
import mysql.connector
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import RandomOverSampler

# Connexion à la base de données
db_connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='wwwiclin_iclinika'
)

# Extraction des données depuis la base de données
query = """
SELECT 
    p.patient_id, 
    p.patient_age, 
    p.patient_allergy, 
    p.patient_bloodgroup, 
    v.vital_bp, 
    v.vital_pulse, 
    v.vital_temp, 
    n.disease, 
    n.stage
FROM 
    patient_opdregistation p
JOIN 
    nurse_vital v ON p.patient_id = v.patient_id
JOIN 
    nurse_diagnosis n ON p.patient_id = n.patient_id
"""

data = pd.read_sql(query, con=db_connection)

# Convertir 'vital_bp' en deux colonnes 'bp_systolic' et 'bp_diastolic'
data[['bp_systolic', 'bp_diastolic']] = data['vital_bp'].str.split('/', expand=True).astype(float)

# Suppression des colonnes non nécessaires
data.drop(['vital_bp'], axis=1, inplace=True)

# Imputation des valeurs manquantes
imputer = SimpleImputer(strategy='most_frequent')
data_imputed = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)

# Conversion des variables catégorielles en numériques
label_encoder = LabelEncoder()
data_imputed['patient_allergy'] = label_encoder.fit_transform(data_imputed['patient_allergy'])
data_imputed['patient_bloodgroup'] = label_encoder.fit_transform(data_imputed['patient_bloodgroup'])
data_imputed['disease'] = label_encoder.fit_transform(data_imputed['disease'])
data_imputed['stage'] = label_encoder.fit_transform(data_imputed['stage'])

# Définir X et y
features = [
    'patient_age', 'patient_bloodgroup', 'patient_allergy',
    'bp_systolic', 'bp_diastolic', 'vital_pulse', 'vital_temp', 'stage'
]
X = data_imputed[features]
y = data_imputed['disease']

# Gestion des classes déséquilibrées avec RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Standardisation des données
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Modèles
models = {
    'RandomForestClassifier': RandomForestClassifier(),
    'GradientBoostingClassifier': GradientBoostingClassifier(),
    'SVC': SVC(),
    'MLPClassifier': MLPClassifier(max_iter=500)
}

# Entraînement et évaluation des modèles
for name, model in models.items():
    model.fit(X_train, y_train)  # Entraîner le modèle
    y_pred = model.predict(X_test)  # Prédictions sur les données de test
    
    # Calculer l'accuracy
    accuracy = accuracy_score(y_test, y_pred)
    
    # Afficher le rapport de classification et l'accuracy
    print(f"Model: {name}")
    print(f"Accuracy: {accuracy:.4f}")
    print(classification_report(y_test, y_pred))


C:\Users\Nessrine\AppData\Local\Temp\ipykernel_7028\1043485800.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, con=db_connection)


Model: RandomForestClassifier
Accuracy: 0.8000
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         1
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         1
          11       1.00      1.00      1.00         1

    accuracy                           0.80         5
   macro avg       0.67      0.67      0.67         5
weighted avg       0.80      0.80      0.80         5



c:\Users\Nessrine\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nessrine\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nessrine\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average

Model: GradientBoostingClassifier
Accuracy: 0.8000
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         1
           4       0.50      1.00      0.67         1
           8       0.00      0.00      0.00         1
          11       1.00      1.00      1.00         1

    accuracy                           0.80         5
   macro avg       0.70      0.80      0.73         5
weighted avg       0.70      0.80      0.73         5

Model: SVC
Accuracy: 0.0000
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       1.0
           2       0.00      0.00      0.00       1.0
           4       0.00      0.00      0.00       1.0
           5       0.00      0.00      0.00       0.0
           6       0.00      0.00      0.00       0.0
           7       0.00      0.00      0.00       0.0
           8       0.00      0.00      0.00       1.0

c:\Users\Nessrine\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nessrine\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nessrine\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

In [3]:
import pandas as pd
import mysql.connector
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import RandomOverSampler
import pickle

# Connexion à la base de données
db_connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='wwwiclin_iclinika'
)

# Extraction des données depuis la base de données
query = """
SELECT 
    p.patient_id, 
    p.patient_age, 
    p.patient_allergy, 
    p.patient_bloodgroup, 
    v.vital_bp, 
    v.vital_pulse, 
    v.vital_temp, 
    n.disease, 
    n.stage
FROM 
    patient_opdregistation p
JOIN 
    nurse_vital v ON p.patient_id = v.patient_id
JOIN 
    nurse_diagnosis n ON p.patient_id = n.patient_id
"""

data = pd.read_sql(query, con=db_connection)

# Convertir 'vital_bp' en deux colonnes 'bp_systolic' et 'bp_diastolic'
data[['bp_systolic', 'bp_diastolic']] = data['vital_bp'].str.split('/', expand=True).astype(float)

# Suppression des colonnes non nécessaires
data.drop(['vital_bp'], axis=1, inplace=True)

# Imputation des valeurs manquantes
imputer = SimpleImputer(strategy='most_frequent')
data_imputed = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)

# Conversion des variables catégorielles en numériques
label_encoder = LabelEncoder()
data_imputed['patient_allergy'] = label_encoder.fit_transform(data_imputed['patient_allergy'])
data_imputed['patient_bloodgroup'] = label_encoder.fit_transform(data_imputed['patient_bloodgroup'])
data_imputed['disease'] = label_encoder.fit_transform(data_imputed['disease'])
data_imputed['stage'] = label_encoder.fit_transform(data_imputed['stage'])

# Définir X et y
features = [
    'patient_age', 'patient_bloodgroup', 'patient_allergy',
    'bp_systolic', 'bp_diastolic', 'vital_pulse', 'vital_temp', 'stage'
]
X = data_imputed[features]
y = data_imputed['disease']

# Gestion des classes déséquilibrées avec RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Standardisation des données
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Modèles
models = {
    'RandomForestClassifier': RandomForestClassifier(),
    'GradientBoostingClassifier': GradientBoostingClassifier(),
    'SVC': SVC(),
    'MLPClassifier': MLPClassifier(max_iter=500),
    'KNeighborsClassifier': KNeighborsClassifier(),
    'DecisionTreeClassifier': DecisionTreeClassifier(),
    'AdaBoostClassifier': AdaBoostClassifier(),
    'XGBClassifier': XGBClassifier()
}

# Entraînement et évaluation des modèles
for name, model in models.items():
    model.fit(X_train, y_train)  # Entraîner le modèle
    y_pred = model.predict(X_test)  # Prédictions sur les données de test
    
    # Calculer l'accuracy
    accuracy = accuracy_score(y_test, y_pred)
    
    # Afficher le rapport de classification et l'accuracy
    print(f"Model: {name}")
    print(f"Accuracy: {accuracy:.4f}")
    print(classification_report(y_test, y_pred))

 # Sauvegarde du modèle
    with open(f'{name}_model.pkl', 'wb') as model_file:
        pickle.dump(model, model_file)

# Sauvegarde du scaler
with open('scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)


C:\Users\Nessrine\AppData\Local\Temp\ipykernel_7028\3472730198.py:44: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, con=db_connection)
c:\Users\Nessrine\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nessrine\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(res

Model: RandomForestClassifier
Accuracy: 0.8000
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         1
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         1
          11       1.00      1.00      1.00         1

    accuracy                           0.80         5
   macro avg       0.67      0.67      0.67         5
weighted avg       0.80      0.80      0.80         5

Model: GradientBoostingClassifier
Accuracy: 0.8000
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         1
           4       0.50      1.00      0.67         1
           8       0.00      0.00      0.00         1
          11       1.00      1.00      1.00         1

    accuracy                     

c:\Users\Nessrine\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nessrine\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nessrine\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

Model: XGBClassifier
Accuracy: 0.8000
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         1
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         1
          11       1.00      1.00      1.00         1

    accuracy                           0.80         5
   macro avg       0.67      0.67      0.67         5
weighted avg       0.80      0.80      0.80         5



c:\Users\Nessrine\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nessrine\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nessrine\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average

Les RandomForestClassifier, DecisionTreeClassifier, et XGBClassifier semblent être les modèles les plus performants parmi ceux testés, avec une exactitude de 80% et des scores de précision pondérée relativement élevés. GradientBoostingClassifier est également un bon candidat mais avec des scores légèrement inférieurs.

Le choix final du modèle dépendra des priorités spécifiques à votre projet, telles que la précision pour certaines classes ou la capacité du modèle à généraliser. Si vous avez des classes spécifiques pour lesquelles la performance est cruciale, vous devrez peut-être ajuster les modèles ou explorer d'autres techniques de traitement des données.

In [4]:
import pandas as pd

# Collecter les résultats
results = []

for name, model in models.items():
    model.fit(X_train, y_train)  # Entraîner le modèle
    y_pred = model.predict(X_test)  # Prédictions sur les données de test
    
    # Calculer l'accuracy
    accuracy = accuracy_score(y_test, y_pred)
    
    # Collecter les résultats pour chaque modèle
    results.append({
        'Model': name,
        'Accuracy': accuracy,
        'Classification Report': classification_report(y_test, y_pred, output_dict=True)
    })

# Convertir les résultats en DataFrame
results_df = pd.DataFrame(results)

# Sauvegarder en CSV
results_df.to_csv('model_results.csv', index=False)


c:\Users\Nessrine\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nessrine\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nessrine\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average

In [5]:
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score

# Entraîner et évaluer le modèle RandomForestClassifier
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

# Calculer l'accuracy
accuracy = accuracy_score(y_test, y_pred)

# Sauvegarder le rapport de classification en CSV
report = classification_report(y_test, y_pred, output_dict=True)

# Convertir le rapport en DataFrame
report_df = pd.DataFrame(report).transpose()
report_df.to_csv('random_forest_report.csv', index=True)


c:\Users\Nessrine\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nessrine\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nessrine\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average

In [6]:
data

,patient_id,patient_age,patient_allergy,patient_bloodgroup,vital_pulse,vital_temp,disease,stage,bp_systolic,bp_diastolic
0,172,28,yes,1,75,36.6,Mental and behavioural disorders,diagnosed,120.0,80.0
1,159,28,yes,6,45,45,Mental and behavioural disorders,diagnosed,34.0,NaN
2,191,34,no,2,72,36.5,Diabetes Mellitus,Type 2,118.0,76.0
3,192,39,yes,3,74,36.7,Hypertension,Stage 1,122.0,78.0
4,193,42,no,4,70,36.4,Asthma,Controlled,119.0,77.0
5,194,26,yes,5,73,36.6,Rheumatoid Arthritis,Active,121.0,79.0
6,195,49,no,6,71,36.3,Hyperlipidemia,Elevated,117.0,75.0
7,196,31,yes,7,76,36.5,Chronic Bronchitis,Stable,120.0,80.0
8,197,54,no,8,74,36.7,Anemia,Mild,118.0,76.0
9,198,44,yes,9,75,36.6,Osteoporosis,Moderate,122.0,78.0


In [1]:
import pickle

model_file = 'random_forest_model.pkl'
try:
    with open(model_file, 'rb') as f:
        model = pickle.load(f)
        print("Modèle chargé avec succès.")
except Exception as e:
    print(f"Erreur lors du chargement du modèle: {e}")


Modèle chargé avec succès.
